In [11]:
#!/usr/bin/env python
# coding: utf-8
import tkinter
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib

def get_main_df():
    #reading the csv, setting the linebreak as 'tab', setting datetime format for the Month index
    #I am setting the index as Month so that we can apply formatting parameters, which will be conserved even as we reset the index a few times within the function
    #i.e. the index will be changed a few times in this function, but at the end it will be set as multi-index for Month and Region, and these parameters will still work then
    main_df = pd.read_csv(Path('data/redfin_table.csv'), on_bad_lines='skip', sep='\t', infer_datetime_format=True, parse_dates=True, index_col=['Month'])
   
    #add state code column using the last two letters from the Region column, and set it as the index. This will be used as a common column for merging the second DF.
    main_df['state_code'] = main_df['Region'].apply(lambda x: x[-2:])
    main_df.reset_index(inplace=True)
    main_df.set_index(keys=['state_code'], inplace=True)
    
    #create second DF for tax rates from CSV and set index to 'state_code'
    tax_df = pd.read_csv("./data/tax_rates_state.csv")
    tax_df = tax_df.drop('state', axis=1)
    tax_df['tax_rate'] = tax_df['tax_rate'].str.replace(r'%', '', regex=True).astype('float')/100
    tax_df= tax_df.query("state_code == ('MA', 'TX', 'CA', 'GA', 'DC', 'FL', 'IN', 'PA', 'CO', 'MI')")
    tax_df = tax_df.set_index(keys='state_code')
    
    #create third df for latlong from csv and set index to 'state_code'
    
    ll_df = pd.read_csv(Path('data/statelatlong.csv'))
    ll_df = ll_df.rename({'State': 'state_code'}, axis='columns')
    ll_df= ll_df.query("state_code == ('MA', 'TX', 'CA', 'GA', 'DC', 'FL', 'IN', 'PA', 'CO', 'MI')")
    ll_df.set_index(keys=['state_code'], inplace=True)
    
    #merge the main and tax DF's using the state_code column as the merge point
    main_df = pd.merge(main_df, tax_df, on='state_code')
    
    #merge the main and latlong DF using the state_code column as the merge point
    main_df = pd.merge(main_df, ll_df, on='state_code')
    
    #reset index again, this time as month and region and for-sale property type
    main_df.reset_index(inplace=True)
    main_df.set_index(keys=['Month', 'Region', 'For-Sale Property Type'], inplace=True)
    main_df.sort_index(level=['Month', 'Region', 'For-Sale Property Type'], ascending=[2, 1, 0], inplace=True)
   
    #drop all rows in the for-sale property type that arent All Residential
    idx=pd.IndexSlice
    main_df = main_df.loc[idx[:,:,['All Residential']]]
                            
    
    #reset index again, this time as month and region; this is the final index format
    main_df.reset_index(inplace=True)
    main_df.set_index(keys=['Month', 'Region'], inplace=True)
    main_df.sort_index(level=['Month', 'Region'], ascending=[1, 0], inplace=True)
    
    #drop unnecessary columns
    main_df = main_df.drop(['For-Sale Property Type', 'state_code', 'City'], axis=1)
    
    #shorten the DF to only include the 10 chosen cities
    main_df = main_df.loc[slice(None), ['Austin, TX', 'San Diego, CA', 'Atlanta, GA', 'Washington, DC', 'West Palm Beach, FL', 'Indianapolis, IN', 'Pittsburgh, PA', 'Detroit, MI', 'Denver, CO', 'Boston, MA'], :]
    
    #clean data of $/%/,
    cols = ['Average Monthly Rent', 'Average Rent YoY', 'Monthly mortgage, 5% down', 'Monthly mortgage, 5% down YoY', 'Monthly mortgage, 20% down', 'Monthly mortgage, 20% down YoY', 'Median sale price', 'Median sale price YoY'] 
    main_df[cols] = main_df[cols].replace({'\$': '', ',': '', '\%': ''}, regex=True)
    
    #set data as int/float where needed
    main_df['Average Monthly Rent'] = main_df['Average Monthly Rent'].astype('int')
    main_df['Monthly mortgage, 5% down'] = main_df['Monthly mortgage, 5% down'].astype('int')
    main_df['Monthly mortgage, 20% down'] = main_df['Monthly mortgage, 20% down'].astype('int')
    main_df['Median sale price']=main_df['Median sale price'].astype('int')
    main_df['Average Rent YoY']=main_df['Average Rent YoY'].astype('float')
    main_df['Monthly mortgage, 5% down YoY']=main_df['Monthly mortgage, 5% down YoY'].astype('float')
    main_df['Monthly mortgage, 20% down YoY']=main_df['Monthly mortgage, 20% down YoY'].astype('float')
    main_df['Median sale price YoY']=main_df['Median sale price YoY'].astype('float')
    
    return main_df
def get_city_list():
    df = get_main_df()
    df.reset_index(inplace=True)
    return df['Region'].unique().tolist()

#the main_df now has a column for tax rates, might not need this function
def get_tax_rates():
    # Return a data frame with columns Region, state_code, tax_rate
    #   for all cities in our main dataframe
    main_df = get_main_df()
    main_df.reset_index(inplace=True)
    df = pd.DataFrame(main_df.Region.unique(), columns=(['Region']))
    df['state_code'] = df['Region'].apply(lambda x: x[-2:])
    tax_df = pd.read_csv("./data/tax_rates_state.csv")
    df_final = pd.merge(df, tax_df, how='right', on=['state_code', 'state_code'])
    df_final.dropna(inplace=True)
    df_final['tax_rate'] = df_final['tax_rate'].str.replace(r'%', '', regex=True).astype('float') / 100
    df_final.reset_index(inplace=True, drop=True)
    return df_final


In [12]:
main_df = get_main_df()
main_df

,,Average Monthly Rent,Average Rent YoY,"Monthly mortgage, 5% down","Monthly mortgage, 5% down YoY","Monthly mortgage, 20% down","Monthly mortgage, 20% down YoY",Median sale price,Median sale price YoY,tax_rate,Latitude,Longitude
Month,Region,,,,,,,,,,,
2021-02-01,"Austin, TX",1644,NaN,1552,NaN,1307,NaN,397184,22.2,0.0180,31.169336,-100.076842
2021-03-01,"Austin, TX",1701,NaN,1728,NaN,1455,NaN,427000,28.4,0.0180,31.169336,-100.076842
2021-04-01,"Austin, TX",1735,NaN,1882,NaN,1584,NaN,466179,43.1,0.0180,31.169336,-100.076842
2021-05-01,"Austin, TX",1824,NaN,1853,NaN,1560,NaN,465000,40.9,0.0180,31.169336,-100.076842
2021-06-01,"Austin, TX",1896,NaN,1918,NaN,1615,NaN,480000,41.2,0.0180,31.169336,-100.076842
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-01,"Boston, MA",3586,19.2,2643,22.9,2225,22.9,599950,9.1,0.0123,42.062940,-71.718067
2022-03-01,"Boston, MA",3672,17.1,2958,27.1,2491,27.1,639000,11.1,0.0123,42.062940,-71.718067
2022-04-01,"Boston, MA",3780,15.1,3435,41.7,2892,41.7,675000,12.4,0.0123,42.062940,-71.718067
